In [16]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/MobileLLM-125M", use_fast=False)
model = AutoModelForCausalLM.from_pretrained("facebook/MobileLLM-125M", trust_remote_code=True)

Some weights of the model checkpoint at facebook/MobileLLM-125M were not used when initializing MobileLLMForCausalLM: ['lm_head.weight']
- This IS expected if you are initializing MobileLLMForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MobileLLMForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MobileLLMForCausalLM were not initialized from the model checkpoint at facebook/MobileLLM-125M and are newly initialized: ['model.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

device='cuda'
question = "what is the llm? explain 1 sentence."
input_ids = tokenizer(question, return_tensors="pt", truncation=True, padding=True).to(device)
model.to(device)

attention_mask=input_ids['attention_mask']
print(attention_mask, input_ids)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [13]:
output = model.generate(input_ids['input_ids'], attention_mask=attention_mask, pad_token_id=tokenizer.eos_token_id, max_new_tokens=50)
print(output)

# 出力のデコード
answer = tokenizer.decode(output[0], skip_special_tokens=True)
print(answer)

AttributeError: 'bool' object has no attribute 'eos_token_id'

In [4]:
question = "a b"
input_ids = tokenizer(question, return_tensors="pt", truncation=True, padding=True).to(device)
output = model(input_ids['input_ids'], attention_mask=attention_mask)
print(output.logits.size())

torch.Size([1, 3, 32000])


In [5]:
import torch
probabilities = torch.softmax(output.logits[0][2], dim=-1)
print(probabilities)
predicted_token_ids = torch.argmax(probabilities, dim=-1)
print(predicted_token_ids)
decoded_text = tokenizer.decode(predicted_token_ids, skip_special_tokens=True)
print(decoded_text)

tensor([1.1364e-05, 3.5737e-05, 1.1140e-05,  ..., 4.4304e-05, 1.6186e-05,
        1.9220e-05], device='cuda:0', grad_fn=<SoftmaxBackward0>)
tensor(26318, device='cuda:0')
disponible
